# Example session

Notebook to try stuff before creating functions

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# select session
import matplotlib as mpl
import seaborn as sns
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from skimage.measure import EllipseModel

import flexiznam as flz

In [ ]:
PROJECT = "hey2_3d-vision_foodres_20220101"
MOUSE = "PZAH6.4b"
SESSION = "S20220419"
RECORDING = "R145152_SpheresPermTubeReward"

In [ ]:
raw_path = Path(flz.PARAMETERS["data_root"]["raw"])
processed_path = Path(flz.PARAMETERS["data_root"]["processed"])
flm_sess = flz.get_flexilims_session(project_id=PROJECT)

In [ ]:
import wayla

wayla.eye_tracking.run_all(
    flexilims_session=flm_sess,
    dlc_model_detect="headfixed_detect_eye",
    dlc_model_tracking="headfixed_track_eye",
    camera_ds_name=f"{MOUSE}_{SESSION}_{RECORDING}_right_eye_camera",
    origin_id=None,
    conflicts="overwrite",
    use_slurm=False,
    dependency=None,
    run_detect=True,
    run_tracking=True,
    run_ellipse=True,
    run_reprojection=True,
    repro_kwargs=None,
)

# Get data

In [ ]:
# get the data
dlc_res, ellipse = analeyesis.get_data(
    camera,
    flexilims_session=flm_sess,
    likelihood_threshold=0.88,
    rsquare_threshold=0.99,
    error_threshold=3,
)
data, sampling = analeyesis.add_behaviour(
    camera, dlc_res, ellipse, speed_threshold=0.01, log_speeds=False
)
assert "valid" in data.columns
data.head()

In [ ]:
# Plot movie with ellipse fit
camera_save_folder = processed_path / camera.path / camera.dataset_name
target_file = camera_save_folder / "eye_tracking_ellipse_overlay.mp4"
video_file = camera.path_full / camera.extra_attributes["video_file"]
dlc_ds_name = "_".join(
    list(camera.genealogy[:-1]) + ["dlc_tracking", camera.dataset_name, "data", "0"]
)
dlc_ds = flz.Dataset.from_flexilims(name=dlc_ds_name, flexilims_session=flm_sess)
cropping = dlc_ds.extra_attributes["cropping"]

start_frame = 19286
vmin = 0
vmax = 150
crop_single_image = True

if False:
    analeyesis.plot_movie(
        camera,
        target_file,
        start_frame=start_frame,
        duration=2,
        dlc_res=dlc_res,
        ellipse=ellipse,
        vmax=vmax,
        vmin=vmin,
        playback_speed=4,
    )
else:
    cam_data = cv2.VideoCapture(str(video_file))
    cam_data.set(cv2.CAP_PROP_POS_FRAMES, start_frame - 1)
    ret, frame = cam_data.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cam_data.release()
    crop_shift = np.array([0, 0])
    plt.figure()
    plt.imshow(gray, cmap="gray")
    if crop_single_image:
        assert cropping is not None
        gray = gray[cropping[2] : cropping[3], cropping[0] : cropping[1]]
    elif cropping is not None:
        crop_shift = np.array([cropping[0], cropping[2]])
    plt.figure()
    plt.imshow(gray, cmap="gray", vmin=vmin, vmax=vmax)
    plt.colorbar()
    track = dlc_res.loc[start_frame]
    track.index = track.index.droplevel(["scorer"])
    xdata = track.loc[[(f"eye_{i}", "x") for i in np.arange(1, 13)]]
    ydata = track.loc[[(f"eye_{i}", "y") for i in np.arange(1, 13)]]
    plt.scatter(xdata + crop_shift[0], ydata + crop_shift[1])
    plt.scatter(
        track.loc[(f"reflection", "x")] + crop_shift[0],
        track.loc[(f"reflection", "y")] + crop_shift[1],
    )

print(crop_shift)

In [ ]:
dlc_data = dlc_res.xs("likelihood", axis="columns", level=2)
dlc_data.columns = dlc_data.columns.droplevel("scorer")
plt.hist(dlc_data["reflection"].iloc[:-3].values[data.valid])
plt.gca().set_xlabel("DLC likelihood")
plt.gcf().set_size_inches(5, 5)
plt.gca().semilogy()

In [ ]:
# plot of fit quality

# DLC likelihood
dlc_data = dlc_res.xs("likelihood", axis="columns", level=2)
dlc_data.columns = dlc_data.columns.droplevel("scorer")
ax = sns.displot(
    dlc_data.drop(
        axis="columns",
        labels=[
            "reflection",
            "left_eye_corner",
            "right_eye_corner",
            "top_eye_lid",
            "bottom_eye_lid",
        ],
    )
)
plt.gca().set_xlabel("DLC likelihood")
plt.gcf().set_size_inches(5, 5)
likelihood_threshold = 0.88
plt.axvline(likelihood_threshold, color="k")
plt.xlim(0.8, 1)
sns.jointplot(data=ellipse[ellipse.valid], x="error", y="rsquare")
sns.jointplot(data=ellipse[ellipse.valid], x="error", y="dlc_avg_likelihood")

# Actual analysis

In [ ]:
fig = plt.figure(figsize=(15, 4))
elli = data[data.valid]
angles = np.rad2deg(elli.angle)
anglim = np.quantile(angles, [0.001, 0.99])
ax = fig.add_subplot(1, 3, 1)
sc = ax.scatter(elli.pupil_x, elli.pupil_y, c=angles, vmax=anglim[1], vmin=anglim[0])
cb = fig.colorbar(ax=ax, mappable=sc)
cb.set_label("Ellipse angle (degrees)")
ax.set_xlabel("Ellipse pupil X (pixels)")
ax.set_ylabel("Ellipse pupil Y (pixels)")
ax.set_aspect("equal")
ax.invert_yaxis()
ax = fig.add_subplot(1, 3, 2)

count, bx, by = np.histogram2d(elli.pupil_x, elli.pupil_y, bins=(70, 70))
h, bx, by = np.histogram2d(elli.pupil_x, elli.pupil_y, weights=angles, bins=(bx, by))
h[count < 1] = np.nan
img = ax.imshow(
    (h / count).T, extent=(bx[0], bx[-1], by[0], by[-1]), vmax=anglim[1], vmin=anglim[0]
)
cb = fig.colorbar(mappable=img, ax=ax)
cb.set_label("Ellipse angle (degrees)")
ax.set_xlabel("Ellipse pupil X (pixels)")
ax.set_ylabel("Ellipse pupil Y (pixels)")

ax = fig.add_subplot(1, 3, 3)
ratio = elli.major_radius / elli.minor_radius
ralim = np.quantile(ratio, [0.01, 0.99])
h, bx, by = np.histogram2d(elli.pupil_x, elli.pupil_y, weights=ratio, bins=(bx, by))
h[count < 1] = np.nan
img = ax.imshow(
    (h / count).T, extent=(bx[0], bx[-1], by[0], by[-1]), vmin=ralim[0], vmax=ralim[1]
)
cb = fig.colorbar(mappable=img, ax=ax)
cb.set_label("Ellipse axes ratio")
ax.set_xlabel("Ellipse pupil X (pixels)")
ax.set_ylabel("Ellipse pupil Y (pixels)")

fig.subplots_adjust(wspace=0.4)

## Check ellipse orientation

To know what is 0 degrees and direction of rotation

In [ ]:
ellipse_model = EllipseModel()

# params are xc, yc, a, b, theta
fig, ax = plt.subplots(1, 1)
fig.set_size_inches(3, 3)
colors = "rgbk"
for ia, angle in enumerate(range(0, 180, 45)):
    ellipse_model.params = (0, 0, 2, 1, np.deg2rad(angle))
    circ_coord = ellipse_model.predict_xy(np.arange(0, 2 * np.pi, 0.1))
    ax.plot(circ_coord[:, 0], circ_coord[:, 1], label=angle, color=colors[ia])
    origin = np.array([0, 0])
    shift = np.array([np.cos(np.deg2rad(angle)), np.sin(np.deg2rad(angle))])
    ax.plot(*[(origin[i], shift[i]) for i in range(2)], color=colors[ia])
    shift_pi = np.array(
        [np.cos(np.deg2rad(angle) + np.pi / 2), np.sin(np.deg2rad(angle) + np.pi / 2)]
    )
    ax.plot(*[(origin[i], shift_pi[i]) for i in range(2)], color=colors[ia], ls="--")
ax.legend(loc="upper left", bbox_to_anchor=(1, 1))
ax.set_aspect("equal")

In [ ]:
# plot time course of eye
fig, axes = plt.subplots(4, 1)
fig.set_size_inches((10, 10))

valid = ellipse.valid
time = np.arange(len(dlc_res)) / sampling
reflection = dlc_res.xs(axis="columns", level=1, key="reflection")
reflection.columns = reflection.columns.droplevel("scorer")
for iax, ax in enumerate(axes):
    for w in ["x", "y"]:
        if iax > 1:
            ax.set_ylabel("Relative to reflection")
            d = (ellipse[f"centre_{w}"] - reflection[w])[valid]
        else:
            ax.set_ylabel("Raw")
            d = ellipse[f"centre_{w}"][valid]
        ax.plot(
            time[valid], d - np.nanmedian(d), label=rf"$\Delta${w.split('_')[0]}", lw=1
        )
axes[0].set_xlim(time[0], time[-1])
axes[2].set_xlim(time[0], time[-1])
for i in [1, 3]:
    axes[i].set_xlim(3000, 3000 + 60 * 2)
    axes[i].set_ylim(-15, 15)
ax.legend(loc="upper right")
ax.set_xlabel("Time (s)")

# Kehr calib

As in Wallace et al.

# Reprojection

Find where is the ellipse given $\theta$ and $\phi$

In [ ]:
phi = np.arange(0, np.pi / 2 + 0.1, np.pi / 3)
theta = np.arange(0 * np.pi / 2, np.pi / 2 + 0.1, np.pi / 8)
plt.figure(figsize=(5, 5))
for p in [np.pi / 6]:
    for t in theta:
        fitted_model = emf.reproj_ellipse(
            phi=p, theta=t, r=1, eye_centre=[150, 200], f_z0=100
        )
        param = list(fitted_model.params)
        # param[1] *=-1
        fitted_model.params = param
        circ_coord = fitted_model.predict_xy(np.arange(0, 2 * np.pi, 0.1))
        label = (np.round(np.rad2deg(p)), np.round(np.rad2deg(t)))
        plt.plot(circ_coord[:, 0], circ_coord[:, 1], label=label, alpha=0.5)
plt.imshow(gray, cmap="gray")
plt.scatter(150, 200)
plt.legend()

## Smooth data

To make fit easier, don't work on individual frames but group frames where the pupil
centre is similar

In [ ]:
# parameters
PLOT = True
CAMERA_DATASET_NAME = camera.full_name
phi0 = 0
theta0 = np.deg2rad(20)

## Bin data

In [ ]:
# make bins of ellipse centre position
print("Bin data", flush=True)
elli = pd.DataFrame(data[data.valid], copy=True)
count, bin_edges_x, bin_edges_y = np.histogram2d(
    elli.pupil_x, elli.pupil_y, bins=(25, 25)
)
elli["bin_id_x"] = bin_edges_x.searchsorted(elli.pupil_x.values)
elli["bin_id_y"] = bin_edges_y.searchsorted(elli.pupil_y.values)
binned_ellipses = elli.groupby(["bin_id_x", "bin_id_y"])
ns = binned_ellipses.valid.aggregate(len)
binned_ellipses = binned_ellipses.aggregate(np.nanmedian)
enough_frames = binned_ellipses[ns > 10]

# PLOT
if PLOT:
    mat = np.zeros((len(ns.index.levels[0]), len(ns.index.levels[1]))) + np.nan
    fig = plt.figure(figsize=(15, 7))
    for ip, p in enumerate(
        ["angle", "minor_radius", "major_radius", "pupil_x", "pupil_y"]
    ):
        mat[
            enough_frames.index.get_level_values(1),
            enough_frames.index.get_level_values(0),
        ] = enough_frames[p]
        lim = np.nanquantile(mat, [0.01, 0.99])
        ax = fig.add_subplot(2, 3, ip + 1)
        img = ax.imshow(mat, vmin=lim[0], vmax=lim[1])
        fig.colorbar(img, ax=ax)
        ax.set_title(p)
    fig.suptitle(CAMERA_DATASET_NAME)

## Estimate eye parameters

In [ ]:
# Find eye centre
print("Find eye centre", flush=True)
p = np.vstack([enough_frames[f"pupil_{a}"].values for a in "xy"])
n = np.vstack([np.cos(enough_frames.angle.values), np.sin(enough_frames.angle.values)])
intercept_minor = emf.pts_intersection(p, n)
n = np.vstack(
    [np.cos(enough_frames.angle + np.pi / 2), np.sin(enough_frames.angle + np.pi / 2)]
)
axes_ratio = enough_frames.minor_radius.values / enough_frames.major_radius.values
eye_centre_binned = intercept_minor.flatten()

delta_pts = (
    np.vstack([enough_frames.pupil_x, enough_frames.pupil_y])
    - eye_centre_binned[:, np.newaxis]
)
sum_sqrt_ratio = np.sum(np.sqrt(1 - axes_ratio**2) * np.linalg.norm(delta_pts, axis=0))
sum_sq_ratio = np.sum(1 - axes_ratio**2)
f_z0_binned = sum_sqrt_ratio / sum_sq_ratio
print(rf"Eye centre: {eye_centre_binned}. f/z0: {f_z0_binned}")
# plot it
if PLOT:
    start_frame = 1000
    video_file = camera.path_full / camera.extra_attributes["video_file"]
    dlc_ds_name = "_".join(
        list(camera.genealogy[:-1]) + ["dlc_tracking", camera.dataset_name, "data", "0"]
    )
    dlc_ds = flz.Dataset.from_flexilims(name=dlc_ds_name, flexilims_session=flm_sess)
    cropping = dlc_ds.extra_attributes["cropping"]
    cam_data = cv2.VideoCapture(str(video_file))
    cam_data.set(cv2.CAP_PROP_POS_FRAMES, start_frame - 1)
    ret, frame = cam_data.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = gray[cropping[2] : cropping[3], cropping[0] : cropping[1]]
    cam_data.release()
    fig, ax = plt.subplots(1, 1)
    fig.set_size_inches(10, 10)
    img = ax.imshow(gray, cmap="gray")
    fig.colorbar(img, ax=ax)
    track = dlc_res.loc[start_frame]
    track.index = track.index.droplevel(["scorer"])

    for i, series in enough_frames.iterrows():
        origin = np.array([series.pupil_x, series.pupil_y])
        ref = np.array([series.reflection_x, series.reflection_y])
        n_v = np.array(
            [np.cos(series.angle + np.pi / 2), np.sin(series.angle + np.pi / 2)]
        )
        rng = np.array([-200, 200])
        ax.plot(
            *[(origin[a] + ref[a] + n_v[a] * rng) for a in range(2)],
            color="purple",
            alpha=0.1,
            lw=1,
        )
    ax.plot(*(eye_centre_binned + ref), color="g", marker="o")
    eye_binned = mpl.patches.Circle(
        xy=(eye_centre_binned + ref),
        radius=f_z0_binned,
        facecolor="none",
        edgecolor="g",
    )
    ax.add_artist(eye_binned)
    ax.set_xlim(0, gray.shape[1])
    _ = ax.set_ylim(gray.shape[0], 0)

In [ ]:
# Example of grid search
# fit median eye position with fine grid
import warnings

p_range = (np.pi / 2, np.pi, 0.5)
grid_size = 30
phi0 = 0
theta0 = np.deg2rad(10)
p0 = (phi0, theta0, 1)
ellipse_params_med = enough_frames.loc[
    :, ["pupil_x", "pupil_y", "major_radius", "minor_radius", "angle"]
].median(axis=0)

grids = [np.linspace(-r, r, grid_size) + p for p, r in zip(p0, p_range)]
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)
    params, ind, errors = emf.grid_search_best_gaze(
        ellipse_params_med,
        eye_centre=eye_centre_binned,
        f_z0=f_z0_binned,
        grid_phi=grids[0],
        grid_theta=grids[1],
        grid_radius=grids[2],
    )
print(f"Initial params: {np.round(p0, 2)}")
print(f"Final params: {np.round(params, 2)}")
img = plt.imshow(
    errors[:, :, ind[2]],
    extent=np.rad2deg((grids[1][0], grids[1][-1], grids[0][0], grids[0][-1])),
    origin="lower",
    cmap="RdBu",
)
plt.colorbar(img)

## Reproject median position

In [ ]:
# fit median eye position with fine grid
print("Fit median position", flush=True)
most_frequent_bin = ns.idxmax()
params_most_frequent_bin = binned_ellipses.loc[
    most_frequent_bin, ["pupil_x", "pupil_y", "major_radius", "minor_radius", "angle"]
]
p0 = (phi0, theta0, 1)
params_med, i, e = emf.minimise_reprojection_error(
    params_most_frequent_bin,
    p0,
    eye_centre_binned,
    f_z0_binned,
    p_range=(np.pi / 2, np.pi / 2, 0.5),
    grid_size=20,
    niter=5,
    reduction_factor=5,
    verbose=True,
)
print(params_med)
phi, theta, radius = params_med
# Plot fit of median position
if PLOT:
    cam_data = cv2.VideoCapture(str(video_file))
    cam_data.set(cv2.CAP_PROP_POS_FRAMES, start_frame - 1)
    ret, frame = cam_data.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = gray[cropping[2] : cropping[3], cropping[0] : cropping[1]]
    cam_data.release()
    ax = plt.subplot(1, 1, 1)
    ax.axis("off")
    ax.imshow(gray, cmap="gray")
    source_model = EllipseModel()
    source_model.params = params_most_frequent_bin
    circ_coord = source_model.predict_xy(np.arange(0, 2 * np.pi, 0.1)) + ref.reshape(
        1, 2
    )
    ax.plot(circ_coord[:, 0], circ_coord[:, 1], label="DLC fit", color="lightblue")
    ax.plot(*(eye_centre_binned + ref), color="g", marker="o", label="Eye centre")
    eye_binned = mpl.patches.Circle(
        xy=(eye_centre_binned + ref),
        radius=f_z0_binned,
        facecolor="none",
        edgecolor="g",
        label=r"$\frac{f}{z_0}$",
    )
    ax.add_artist(eye_binned)

    fitted_model = emf.reproj_ellipse(
        phi=phi, theta=theta, r=radius, eye_centre=eye_centre_binned, f_z0=f_z0_binned
    )
    circ_coord = fitted_model.predict_xy(np.arange(0, 2 * np.pi, 0.1)) + ref.reshape(
        1, 2
    )
    ax.plot(
        circ_coord[:, 0],
        circ_coord[:, 1],
        label="Reprojection",
        color="purple",
        ls="--",
    )
    ax.legend(loc="upper left", bbox_to_anchor=(1, 1))
    plt.tight_layout()

### Also all binned position

In [ ]:
# optimise for all binned positions
print("Reproject binned data", flush=True)
eye_rotation_initial = np.zeros((len(enough_frames), 3))
grid_angles = np.deg2rad(np.arange(0, 360, 5))
grid_radius = np.arange(0.8, 1.2, 0.1)
for i_pos, (pos, s) in enumerate(enough_frames.iterrows()):
    ellipse_params = s[["pupil_x", "pupil_y", "major_radius", "minor_radius", "angle"]]
    p, i, e = emf.minimise_reprojection_error(
        ellipse_params,
        p0=params_med,
        eye_centre=eye_centre_binned,
        f_z0=f_z0_binned,
        p_range=(np.pi / 3, np.pi / 3, 0.5),
        grid_size=10,
        niter=5,
        reduction_factor=5,
        verbose=False,
    )
    eye_rotation_initial[i_pos] = p

In [ ]:
if PLOT:
    mat = (
        np.zeros(
            (
                len(binned_ellipses.index.levels[0]),
                len(binned_ellipses.index.levels[1]),
                3,
            )
        )
        + np.nan
    )
    for i_pos, (pos, _) in enumerate(enough_frames.iterrows()):
        mat[pos[1], pos[0]] = eye_rotation_initial[i_pos]
    fig = plt.figure(figsize=(15, 6))
    labels = ["phi", "theta", "radius"]
    v = binned_ellipses[["pupil_x", "pupil_y"]].values
    lims = np.vstack([np.nanmin(v, axis=0), np.nanmax(v, axis=0)]) + ref

    for i in range(3):
        plt.subplot(1, 3, 1 + i)
        if i < 2:
            d = np.rad2deg(mat[..., i])
        else:
            d = mat[..., i]
        plt.imshow(gray, cmap="gray")
        plt.imshow(d, extent=np.hstack([lims[:, 0], lims[::-1, 1]]))
        plt.title(labels[i])
        plt.colorbar()
        plt.xlim(0, gray.shape[1])
        plt.ylim(gray.shape[0], 0)

## Optimise eye parameters

In [ ]:
# Now optimise eye_centre and f_z0
print("Optimise eye parameters", flush=True)
# use a quarter of the frames to go a bit faster
skip = int(np.ceil(len(enough_frames) / 20))
source_ellipses = (
    enough_frames[::skip]
    .loc[:, ["pupil_x", "pupil_y", "major_radius", "minor_radius", "angle"]]
    .values
)
gazes = eye_rotation_initial[::skip]
(x, y, f_z0), ind, err = emf.optimise_eye_parameters(
    ellipses=source_ellipses,
    gazes=gazes,
    p0=(*eye_centre_binned, f_z0_binned),
    p_range=(70, 70, 50),
    grid_size=7,
    niter=3,
    reduction_factor=3,
    verbose=True,
    inner_search_kwargs=dict(
        p_range=(np.deg2rad(20), np.deg2rad(20), 0.4), grid_size=5, niter=3
    ),
)
eye_centre = np.array([x, y])

## Refit median

In [ ]:
# Refit median eye position with new eye
# needed because we want to limit the search 60 degrees around that
params_med, i, e = emf.minimise_reprojection_error(
    params_most_frequent_bin,
    params_med,
    eye_centre,
    f_z0,
    p_range=(np.pi / 3, np.pi / 3, 0.5),
    grid_size=20,
    niter=5,
    reduction_factor=5,
    verbose=True,
)

if PLOT:
    # replot median eye posisiton with better eye
    ax = plt.subplot(1, 1, 1)
    ax.axis("off")
    ax.imshow(gray, cmap="gray")
    source_model = EllipseModel()
    source_model.params = params_most_frequent_bin
    circ_coord = source_model.predict_xy(np.arange(0, 2 * np.pi, 0.1)) + ref.reshape(
        1, 2
    )
    ax.scatter(*(eye_centre_binned + ref), color="g", label="Eye centre (initial)")
    ax.scatter(
        *(eye_centre + ref), ec="purple", label="Eye centre (optimised)", fc="None"
    )
    eye_binned = mpl.patches.Circle(
        xy=(eye_centre_binned + ref),
        radius=f_z0_binned,
        facecolor="none",
        edgecolor="g",
        label=r"$\frac{f}{z_0}$ (initial)",
    )
    ax.add_artist(eye_binned)

    eye_binned = mpl.patches.Circle(
        xy=(eye_centre + ref),
        radius=f_z0,
        facecolor="none",
        edgecolor="purple",
        ls="--",
        label=r"$\frac{f}{z_0}$ (optimised)",
    )
    ax.add_artist(eye_binned)

    ax.plot(circ_coord[:, 0], circ_coord[:, 1], label="DLC fit", color="lightblue")
    # use phi/theta/radius to get original estimate
    fitted_model = emf.reproj_ellipse(
        *(phi, theta, radius), eye_centre=eye_centre_binned, f_z0=f_z0_binned
    )
    circ_coord = fitted_model.predict_xy(np.arange(0, 2 * np.pi, 0.1)) + ref.reshape(
        1, 2
    )
    ax.plot(
        circ_coord[:, 0],
        circ_coord[:, 1],
        label="Original reprojection",
        color="orange",
        ls="--",
    )

    fitted_model = emf.reproj_ellipse(*params_med, eye_centre=eye_centre, f_z0=f_z0)
    circ_coord = fitted_model.predict_xy(np.arange(0, 2 * np.pi, 0.1)) + ref.reshape(
        1, 2
    )
    ax.plot(
        circ_coord[:, 0],
        circ_coord[:, 1],
        label="Optimised reprojection",
        color="purple",
        ls=":",
    )
    ax.legend(loc="upper left", bbox_to_anchor=(1, 1))
    plt.tight_layout()

# Fit gaze

for all frames

In [ ]:
# optimise for all frames

eye_rotation_binned = np.zeros((len(binned_ellipses), 3)) + np.nan

for i_pos, (pos, series) in enumerate(binned_ellipses.iterrows()):
    if np.mod(i_pos, 50) == 0:
        print(i_pos)
    if not series.valid:
        continue
    ellipse_params = series[
        ["pupil_x", "pupil_y", "major_radius", "minor_radius", "angle"]
    ]
    pa, i, e = emf.minimise_reprojection_error(
        ellipse_params,
        p0=params_med,
        eye_centre=eye_centre,
        f_z0=f_z0,
        p_range=(np.pi / 3, np.pi / 3, 0.5),
        grid_size=10,
        niter=3,
        reduction_factor=5,
        verbose=False,
    )
    eye_rotation_binned[i_pos] = pa

# Get all gaze fits

In [ ]:
camera_save_folder

In [ ]:
import os

f = os.listdir(camera_save_folder)
batch_eye_params = np.load(
    camera_save_folder / f"{camera.dataset_name}_eye_parameters.npz"
)
eye_rotation_all = np.load(
    camera_save_folder / f"{camera.dataset_name}_eye_rotation_by_frame.npy"
)
med_pos_deg = np.rad2deg(batch_eye_params["median_eye_position_parameters"][:2])
plt.hist(
    np.rad2deg(eye_rotation_all[:, 0]),
    bins=np.arange(-100, 100, 5),
    label="Phi",
    color="orange",
)
plt.hist(
    np.rad2deg(eye_rotation_all[:, 1]),
    bins=np.arange(-100, 100, 5),
    label="Theta",
    color="purple",
)
plt.axvline(med_pos_deg[0], color="orange")
plt.axvline(med_pos_deg[1], color="purple")

plt.legend()

# Transform to world coordinates

In [ ]:
# get calibration data
calibration_folder = processed_path / PROJECT / "Calibrations"

calib_data = dict()
for cam_name in ["RightEyeCam", "LeftEyeCam"]:
    calib_data[cam_name.lower()] = dict()
    folder = calibration_folder / cam_name
    folder = list(folder.glob("*xtrinsics_flat"))[0]  # case is inconsistent
    folder = folder / "20220818" / "aruco5_5mm"
    assert folder.exists()
    for trial in folder.glob("trial*"):
        fname = str(trial / "camera_extrinsics_flat.yml")
        s = cv2.FileStorage()
        s.open(fname, cv2.FileStorage_READ)
        rvec = s.getNode("rvec").mat()
        tvec = s.getNode("tvec").mat()
        calib_data[cam_name.lower()][trial.name] = dict(rvec=rvec, tvec=tvec)
# take median across trials
extrinsics = dict()
for cam, trials in calib_data.items():
    extrinsics[cam] = dict()
    for w in ["rvec", "tvec"]:
        extrinsics[cam][w] = np.median(
            np.vstack([d[w].flatten() for d in trials.values()]), axis=0
        )
extrinsics[cam]
# get the camera we need for this acq and build tform matrix
extrin = extrinsics[camera.dataset_name.replace("_", "")[:-3]]
tform = np.zeros((4, 4))
tform[3, 3] = 1
rmat, jac = cv2.Rodrigues(extrin["rvec"])
tform[:3, :3] = rmat
tform[:3, 3] = extrin["tvec"]
print(np.round(tform, 2))

In [ ]:
# get gaze vectors
gaze_vec = np.vstack([emf.get_gaze_vector(p[0], p[1]) for p in eye_rotation_binned])

fig = plt.figure(figsize=(16, 4))
ax = fig.add_subplot(1, 3, 1, projection="3d")
ax.scatter(0, 0, 0, color="k")
ax.scatter(gaze_vec[:, 0], gaze_vec[:, 1], gaze_vec[:, 2], c=eye_rotation_binned[:, 0])
ax.set_xlabel("Camera X")
ax.set_ylabel("Camera Y")
ax.set_zlabel("Camera Z")
ax = fig.add_subplot(1, 3, 2)
sc = ax.scatter(gaze_vec[:, 0], gaze_vec[:, 1], c=np.rad2deg(eye_rotation_binned[:, 0]))
ax.set_xlabel("Camera X")
ax.set_ylabel("Camera Y")
cb = fig.colorbar(sc, ax=ax)
cb.set_label("Phi")
ax = fig.add_subplot(1, 3, 3)
ax.set_xlabel("Camera X")
ax.set_ylabel("Camera Y")
sc = ax.scatter(gaze_vec[:, 0], gaze_vec[:, 1], c=np.rad2deg(eye_rotation_binned[:, 1]))
cb = fig.colorbar(sc, ax=ax)
cb.set_label("Theta")

In [ ]:
rmat.shape

In [ ]:
# rotate gaze vectors
def convert_to_world(gaze_vec):
    flipped_gaze = np.array(gaze_vec, copy=True)
    flipped_gaze[:, 1] *= 1  # to have back y going up instead of down
    rotated_gaze_vec = (rmat @ flipped_gaze.T).T
    rotated_gaze_vec = rotated_gaze_vec[
        :, [0, 2, 1]
    ]  # because of camera mirror made it a lefthand coordinate system
    return rotated_gaze_vec


def gaze_to_azel(gaze_vector, zero_median=False):
    azimuth = np.arctan2(gaze_vector[:, 1], gaze_vector[:, 0])
    elevation = np.arctan2(gaze_vector[:, 2], np.sum(gaze_vector[:, :2] ** 2, axis=1))
    # zero the median pos
    if zero_median:
        azimuth -= np.nanmedian(azimuth)
        elevation -= np.nanmedian(elevation)
        # put back in -pi pi
        azimuth = np.mod(azimuth + np.pi, 2 * np.pi) - np.pi
        elevation = np.mod(elevation + np.pi, 2 * np.pi) - np.pi
    return azimuth, elevation


rotated_gaze_vec = emf.convert_to_world(gaze_vec, rmat=rmat)
azimuth, elevation = emf.gaze_to_azel(rotated_gaze_vec)

fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(2, 3, 1, projection="3d")
ax.set_xlabel("World X")
ax.set_ylabel("World Y")
ax.set_zlabel("World Z")

ax.scatter(0, 0, 0, color="k")
ax.scatter(
    rotated_gaze_vec[:, 0],
    rotated_gaze_vec[:, 1],
    rotated_gaze_vec[:, 2],
    c=eye_rotation_binned[:, 0],
)
ax = fig.add_subplot(2, 5, 3, aspect="equal")
sc = ax.scatter(rotated_gaze_vec[:, 0], rotated_gaze_vec[:, 1], c=np.rad2deg(elevation))
ax.set_xlabel("World X")
ax.set_ylabel("World Y")
cb = fig.colorbar(sc, ax=ax)
cb.set_label("El")
ax = fig.add_subplot(2, 5, 4, aspect="equal")
ax.set_xlabel("World X")
ax.set_ylabel("World Y")
sc = ax.scatter(rotated_gaze_vec[:, 0], rotated_gaze_vec[:, 1], c=np.rad2deg(azimuth))
cb = fig.colorbar(sc, ax=ax)
cb.set_label("Az")
ax = fig.add_subplot(2, 5, 5, aspect="equal")
ax.set_xlabel("World X")
ax.set_ylabel("World Z")
sc = ax.scatter(rotated_gaze_vec[:, 0], rotated_gaze_vec[:, 2], c=np.rad2deg(elevation))
cb = fig.colorbar(sc, ax=ax)
cb.set_label("El")

ax = fig.add_subplot(2, 2, 3, aspect="equal")
ax.set_xlabel("World Y")
ax.set_ylabel("World Z")
sc = ax.scatter(rotated_gaze_vec[:, 1], rotated_gaze_vec[:, 2], c=np.rad2deg(azimuth))
cb = fig.colorbar(sc, ax=ax)
cb.set_label("Az")


ax = fig.add_subplot(2, 2, 4, aspect="equal")
ax.set_xlabel("World Y")
ax.set_ylabel("World Z")
sc = ax.scatter(rotated_gaze_vec[:, 1], rotated_gaze_vec[:, 2], c=np.rad2deg(elevation))
cb = fig.colorbar(sc, ax=ax)
cb.set_label("El")

In [ ]:
gaze_vec_all = np.vstack([emf.get_gaze_vector(p[0], p[1]) for p in eye_rotation_all])
flipped_gaze = np.array(gaze_vec, copy=True)
flipped_gaze[:, 0] *= -1
rotated_gaze = (rmat @ flipped_gaze.T).T
flipped_gaze_all = np.array(gaze_vec_all, copy=True)
flipped_gaze_all[:, 0] *= -1
rotated_gaze_all = (rmat @ gaze_vec_all.T).T

valid = ~np.isnan(gaze_vec[:, 0])
plt.figure(figsize=(10, 10))
plt.subplot(2, 2, 1)
plt.hist(np.rad2deg(gaze_vec[valid]), histtype="step")
plt.title("Gaze vector in eye/camera coordinates")
plt.subplot(2, 2, 2)
_ = plt.hist(np.rad2deg(rotated_gaze[valid]), histtype="step")
plt.title("Gaze vector in world coordinates")
plt.subplot(2, 2, 3)
plt.hist(
    np.rad2deg(gaze_vec[valid] - np.nanmedian(gaze_vec[valid], axis=0)), histtype="step"
)
plt.subplot(2, 2, 4)
_ = plt.hist(
    np.rad2deg(rotated_gaze[valid] - np.nanmedian(rotated_gaze[valid], axis=0)),
    histtype="step",
)

In [ ]:
azimuth, elevation = gaze_to_azel(rotated_gaze, zero_median=True)
azimuth_all, elevation_all = gaze_to_azel(rotated_gaze_all, zero_median=True)

bins = np.arange(-50, 50)
valid = ~np.isnan(rotated_gaze_all[:, 0])
plt.subplot(2, 1, 1)
plt.hist(
    np.rad2deg(eye_rotation_all[valid, 0]),
    bins=bins,
    histtype="step",
    label="Phi",
)
plt.hist(
    np.rad2deg(eye_rotation_all[valid, 1]),
    bins=bins,
    histtype="step",
    label="Theta",
)

plt.legend(loc=0)
plt.subplot(2, 1, 2)
plt.hist(np.rad2deg(azimuth_all[valid]), bins=bins, histtype="step", label="Azimuth")
plt.hist(
    np.rad2deg(elevation_all[valid]), bins=bins, histtype="step", label="Elevation"
)
plt.legend(loc=0)

In [ ]:
fig = plt.figure(figsize=(10, 10))
titl = ["Camera coordinates", "World coordinates"]
valid = ~np.isnan(gaze_vec[:, 0])
for iax, vec in enumerate([gaze_vec, rotated_gaze]):
    for ia in range(2):
        ax = fig.add_subplot(2, 2, 1 + iax * 2 + ia, projection="3d")
        if not ia:
            ax.set_title(titl[iax])
        ax.set_xlabel("X")
        ax.set_ylabel("Y")
        ax.set_zlabel("Z")
        for g in vec[valid]:
            ax.plot(*[(0, i) for i in g], color="k")
        if not iax:
            c = eye_rotation_binned[valid, 0] if ia else eye_rotation_binned[valid, 1]
            l = "Phi" if ia else "Theta"
        else:
            c = azimuth if ia else elevation
            l = "Azimuth" if ia else "Elevation"
        sc = ax.scatter(*vec[valid].T, c=np.rad2deg(c[valid]))
        cb = fig.colorbar(sc, ax=ax)
        cb.set_label(l)

In [ ]:
c = np.rad2deg(azimuth[valid])
plt.subplot(3, 2, 1, aspect="equal")
plt.scatter(0, 0, color="k")
plt.scatter(gaze_vec[valid, 0], gaze_vec[valid, 1], c=c)
plt.subplot(3, 2, 2, aspect="equal")
plt.scatter(0, 0, color="k")
plt.scatter(rotated_gaze[valid, 0], rotated_gaze[valid, 1], c=c)
plt.subplot(3, 2, 3, aspect="equal")
plt.scatter(0, 0, color="k")
plt.scatter(gaze_vec[valid, 0], gaze_vec[valid, 2], c=c)
plt.subplot(3, 2, 4, aspect="equal")
plt.scatter(0, 0, color="k")
plt.scatter(rotated_gaze[valid, 0], rotated_gaze[valid, 2], c=c)
plt.subplot(3, 2, 5, aspect="equal")
plt.scatter(0, 0, color="k")
plt.scatter(gaze_vec[valid, 1], gaze_vec[valid, 2], c=c)
plt.subplot(3, 2, 6, aspect="equal")
plt.scatter(0, 0, color="k")
plt.scatter(rotated_gaze[valid, 1], rotated_gaze[valid, 2], c=c)

In [ ]:
plt.subplot(2, 1, 1)
plt.scatter(*np.rad2deg(eye_rotation_binned[:, :2].T), c=eye_rotation_binned[:, 2])
plt.gca().set_aspect("equal")
plt.xlabel("Phi")
plt.ylabel("Theta")
plt.subplot(2, 1, 2)
plt.scatter(np.rad2deg(azimuth), np.rad2deg(elevation), c=eye_rotation_binned[:, 2])
plt.gca().set_aspect("equal")
plt.xlabel("Azimuth")
plt.ylabel("Elevation")

In [ ]:
mat = (
    np.zeros(
        (len(binned_ellipses.index.levels[0]), len(binned_ellipses.index.levels[1]), 2)
    )
    + np.nan
)
for i_pos, (pos, _) in enumerate(binned_ellipses.iterrows()):
    mat[pos[0], pos[1]] = [azimuth[i_pos], elevation[i_pos]]
fig = plt.figure(figsize=(15, 10))
labels = ["azimuth", "elevation"]
v = binned_ellipses[["pupil_x", "pupil_y"]].values
lims = np.vstack([np.nanmin(v, axis=0), np.nanmax(v, axis=0)]) + ref
plt.imshow(gray, cmap="gray")
for i, v in enumerate([azimuth, elevation]):
    ax = plt.subplot(1, 2, 1 + i)
    plt.imshow(gray, cmap="gray")
    plt.imshow(
        np.rad2deg(mat[..., i]),
        extent=np.hstack([lims[:, 0], lims[::-1, 1]]),
        cmap="RdBu",
        vmin=-20,
        vmax=20,
    )
    plt.title(labels[i])
    plt.colorbar()
    ax.set_xlim([0, gray.shape[1]])
    ax.set_ylim([gray.shape[0], 0])

In [ ]:
source_model = EllipseModel()
source_model.params = data.loc[
    start_frame, ["pupil_x", "pupil_y", "major_radius", "minor_radius", "angle"]
].values
circ_coord = source_model.predict_xy(np.arange(0, 2 * np.pi, 0.1)) + ref.reshape(1, 2)
# refit

params_start_frame, i, e = emf.minimise_reprojection_error(
    source_model,
    p0=params_med,
    eye_centre=eye_centre,
    f_z0=f_z0,
    p_range=(np.pi / 3, np.pi / 3, 0.5),
    grid_size=20,
    niter=5,
    reduction_factor=5,
    verbose=True,
)
fitted_model = emf.reproj_ellipse(*params_start_frame, eye_centre=eye_centre, f_z0=f_z0)

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig = plt.figure(figsize=(17.7165, 7.87402))

ax = plt.subplot2grid((5, 5), (4, 4))

# Eye tracking video example
ax = plt.subplot2grid((5, 6), (0, 0), rowspan=2, colspan=2)
labels = ["azimuth", "elevation"]
v = binned_ellipses[["pupil_x", "pupil_y"]].values
lims = np.vstack([np.nanmin(v, axis=0), np.nanmax(v, axis=0)]) + ref
circ_coord = fitted_model.predict_xy(np.arange(0, 2 * np.pi, 0.1)) + ref.reshape(1, 2)
mat = (
    np.zeros(
        (len(binned_ellipses.index.levels[0]), len(binned_ellipses.index.levels[1]), 2)
    )
    + np.nan
)
for i_pos, (pos, _) in enumerate(binned_ellipses.iterrows()):
    mat[pos[0], pos[1]] = [azimuth[i_pos], elevation[i_pos]]

ax.imshow(gray, cmap="gray", vmin=20, vmax=100, zorder=-1)
img = ax.imshow(
    np.rad2deg(mat[..., 0]),
    extent=np.hstack([lims[:, 0], lims[::-1, 1]]),
    cmap="RdBu_r",
    vmin=-20,
    vmax=20,
    zorder=10,
)
ax.plot(
    circ_coord[:, 0],
    circ_coord[:, 1],
    label="Reprojection",
    color="lightblue",
    alpha=0.5,
    zorder=5,
)
pupil_c = np.array(fitted_model.params[:2])
ax.plot(*(eye_centre + ref), marker="o", ms=5, mfc="k", mec="none", zorder=1)
ax.plot(*(pupil_c + ref), marker="o", ms=5, mfc="none", color="lightblue", alpha=0.5)
ax.plot(
    *[(np.array([eye_centre[i], pupil_c[i]]) + ref[i]) for i in range(2)],
    color="lightblue",
    zorder=2,
    alpha=0.5
)
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
cb = fig.colorbar(img, cax=cax)
cb.set_label("Azimuth (degrees)")
ax.set_xlim([gray.shape[1], 0])
ax.set_ylim([gray.shape[0] - 80, 50])
ax.text(x=0.55, y=0.8, s="Dorsal", color="white", transform=ax.transAxes)
ax.text(x=0.65, y=0.1, s="Nasal", color="white", transform=ax.transAxes)
ax.axis("off")

ax_timecourse = plt.subplot2grid((5, 6), (0, 2), rowspan=1, colspan=2)
n = 6000
b = 16000
time = np.arange(n) / sampling
colors = ["purple", "orange"]
ax_timecourse.plot(
    time, data.pupil_x.iloc[b : b + n] - np.nanmedian(data.pupil_x), color=colors[0]
)
ax_timecourse.plot(
    time, data.pupil_y.iloc[b : b + n] - np.nanmedian(data.pupil_y), color=colors[1]
)
ax_timecourse.set_xlabel("Time (s)")
ax_timecourse.set_ylabel(r"$\Delta$angle (degrees)")
ax_timecourse.set_xlim(0, time.max())
ax_timecourse.set_xticks(np.arange(0, time.max(), 60))
ax_timecourse.spines["top"].set_visible(False)
ax_timecourse.spines["right"].set_visible(False)
divider = make_axes_locatable(ax_timecourse)
ax = divider.append_axes("right", size="10%", pad=0.05)
bins = np.arange(-25, 25)
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.hist(
    data.pupil_x - np.nanmedian(data.pupil_x),
    orientation="horizontal",
    density=False,
    bins=bins,
    color=colors[0],
    histtype="step",
    lw=2,
)
ax.hist(
    data.pupil_y - np.nanmedian(data.pupil_y),
    orientation="horizontal",
    density=False,
    bins=bins,
    color=colors[1],
    histtype="step",
    lw=2,
)
ax.set_xlabel("# frames")
ax.set_yticks([])

# Depth analysis

In [ ]:
import matplotlib as mpl
from matplotlib import cm

data["dx"] = data.pupil_x.diff()
data["dy"] = data.pupil_y.diff()
data["d_med"] = 0

depth_list = np.unique(data.depth)
cmap = cm.cool.reversed()
line_colors = []
norm = mpl.colors.Normalize(vmin=np.log(min(depth_list)), vmax=np.log(max(depth_list)))
col_dict = dict()
for depth in depth_list:
    rgba_color = cmap(norm(np.log(depth)), bytes=True)
    rgba_color = tuple(it / 255 for it in rgba_color)
    line_colors.append(rgba_color)
    col_dict[depth] = rgba_color

fig, axes = plt.subplots(3, 1)
fig.set_size_inches(6, 10)
labels = ["X position", "Y position", "Distance to median position"]
d = data[(~np.isnan(data.dx)) & (~np.isnan(data.depth))]
for iw, w in enumerate(["dx", "dy", "d_med"]):
    sns.violinplot(data=d, x="depth", y=w, palette=line_colors, ax=axes[iw])
    axes[iw].set_ylabel(labels[iw])

In [ ]:
import matplotlib as mpl
from matplotlib import cm

depth_list = np.unique(data.depth)
cmap = cm.cool.reversed()
line_colors = []
norm = mpl.colors.Normalize(vmin=np.log(min(depth_list)), vmax=np.log(max(depth_list)))
col_dict = dict()
for depth in depth_list:
    rgba_color = cmap(norm(np.log(depth)), bytes=True)
    rgba_color = tuple(it / 255 for it in rgba_color)
    line_colors.append(rgba_color)
    col_dict[depth] = rgba_color

fig, axes = plt.subplots(3, 1)
fig.set_size_inches(5, 7)
labels = ["Motion", "Small movements", "Saccades"]
data["saccade"] = data.mvt > 5
d = data[(~np.isnan(data.dx)) & (~np.isnan(data.depth))]
for iw in range(2):
    if not iw:
        sns.violinplot(data=d, x="depth", y="mvt", palette=line_colors, ax=axes[iw])
    else:
        sns.violinplot(
            data=d[d.mvt < 2], x="depth", y="mvt", palette=line_colors, ax=axes[iw]
        )
    axes[iw].set_ylabel(labels[iw])

sac_per_depth = d.groupby("depth").saccade.aggregate(np.nansum)
sample_per_depth = d.groupby("depth").saccade.aggregate(len)

axes[2].bar(
    x=np.arange(len(sac_per_depth)),
    height=sac_per_depth / sample_per_depth * sampling,
    color=line_colors,
)
axes[2].set_xticks(np.arange(len(sac_per_depth)))
axes[2].set_xticklabels(sac_per_depth.index)
axes[2].set_ylabel("Saccades per second")

In [ ]:
import matplotlib as mpl
from matplotlib import cm

lr = np.log10(data.rs)
data["running_bin"] = np.round(data.rs / 10) * 10

running_bins = np.unique(data[data.valid]["running_bin"])
cmap = cm.viridis
rs_colors = []
norm = mpl.colors.Normalize(vmin=min(running_bins), vmax=max(running_bins))
for rb in running_bins:
    rgba_color = cmap(norm(rb), bytes=True)
    rgba_color = tuple(it / 255 for it in rgba_color)
    rs_colors.append(rgba_color)


fig, axes = plt.subplots(3, 1)
fig.set_size_inches(5, 7)
labels = ["Motion", "Small movements", "Saccades"]
data["saccade"] = data.mvt > 5
d = data[(~np.isnan(data.dx)) & (~np.isnan(data.depth))]
for iw in range(2):
    if not iw:
        sns.violinplot(data=d, x="running_bin", y="mvt", palette="viridis", ax=axes[iw])
    else:
        sns.violinplot(
            data=d[d.mvt < 2], x="running_bin", y="mvt", palette="viridis", ax=axes[iw]
        )
    axes[iw].set_ylabel(labels[iw])

sac_per_depth = d.groupby("running_bin").saccade.aggregate(np.nansum)
sample_per_depth = d.groupby("running_bin").saccade.aggregate(len)

axes[2].bar(
    x=np.arange(len(sac_per_depth)),
    height=sac_per_depth / sample_per_depth * sampling,
    color=rs_colors,
)
axes[2].set_xticks(np.arange(len(sac_per_depth)))
axes[2].set_xticklabels(np.array(sac_per_depth.index, dtype=int))
axes[2].set_ylabel("Saccades per second")

In [ ]:
from scipy.stats import mannwhitneyu

depth_list = np.unique(d.depth)
props = ["dx", "dy", "d_med"]
pval_mat = np.zeros([len(props)] + [len(depth_list)] * 2)

for ix, dx in enumerate(depth_list):
    xdf = d[d.depth == dx]
    for iy, dy in enumerate(depth_list):
        for ip, p in enumerate(props):
            ydf = d[d.depth == dy]
            if ix == iy:
                pval_mat[ip, ix, iy] = 0
            else:
                w = mannwhitneyu(xdf[p].values, ydf[p].values)
                pval_mat[ip, ix, iy] = w.pvalue

fig, axes = plt.subplots(1, 3)
for ip, p in enumerate(props):
    axes[ip].imshow(pval_mat[ip] - 0.05, cmap="RdBu", origin="lower")

In [ ]:
fig, axes = plt.subplots(1, 2)
for d, ddf in data.groupby("depth"):
    axes[0].errorbar(
        x=np.nanmedian(ddf.dx),
        y=np.nanmedian(ddf.dy),
        xerr=np.nanstd(ddf.dx),
        yerr=np.nanstd(ddf.dy),
        label=int(d),
        marker="o",
        color=col_dict[d],
    )
    axes[1].errorbar(
        x=np.nanmean(ddf.dx),
        y=np.nanmean(ddf.dy),
        xerr=np.nanstd(ddf.dx) / np.sqrt(np.sum(~np.isnan(ddf.dx))),
        yerr=np.nanstd(ddf.dy) / np.sqrt(np.sum(~np.isnan(ddf.dy))),
        label=int(d),
        marker=".",
        lw=3,
        color=col_dict[d],
    )

for ax in axes:
    ax.set_aspect("equal")
ax.legend(loc="upper left", bbox_to_anchor=(1, 1))
axes[0].set_title("Eye position (median +/- std)")
axes[1].set_title("Eye position (mean +/- std)")

In [ ]:
img_VS = pd.merge_asof(
    img_VS,
    mousez_logger,
    on="HarpTime",
    allow_exact_matches=True,
    direction="backward",
)

img_VS.EyeZ = img_VS.EyeZ / 100  # Convert cm to m
img_VS.MouseZ = img_VS.MouseZ / 100  # Convert cm to m
img_VS.Depth = img_VS.Depth / 100  # Convert cm to m
img_VS.Z0 = img_VS.Z0 / 100  # Convert cm to m

depth_list = img_VS["Depth"].unique()
depth_list = np.round(depth_list, 2)
depth_list = depth_list[~np.isnan(depth_list)].tolist()
depth_list.remove(-99.99)
depth_list.sort()

In [ ]:
import pickle

# print(img_VS[:20], flush=True)
# Save img_VS
with open(protocol_folder / "img_VS.pickle", "wb") as handle:
    pickle.dump(img_VS, handle, protocol=pickle.HIGHEST_PROTOCOL)
print("Timestamps aligned and saved.", flush=True)
print("---STEP 3 FINISHED.---", "\n", flush=True)

# -----STEP4: Get the visual stimulation structure and Save (find the imaging frames for visual stimulation)-----
print("---START STEP 4---", "\n", "Get vis-stim structure...", flush=True)
with open(protocol_folder / "img_VS.pickle", "rb") as handle:
    img_VS = pickle.load(handle)
from cottage_analysis.stimulus_structure import sphere_structure as vis_stim_structure

stim_dict = vis_stim_structure.create_stim_dict(
    depth_list=depth_list, img_VS=img_VS, choose_trials=None
)

In [ ]:
img_VS.head()

In [ ]:
img_VS.shape

In [ ]:
dlc_res.shape